In [15]:
import optuna
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import ast

# Define the DNN model
class DNN(nn.Module):
    def __init__(self, input_dim, hidden_dims, output_dim, dropout_rate, activation_fn):
        super(DNN, self).__init__()
        layers = []
        layers.append(nn.Linear(input_dim, hidden_dims[0]))
        layers.append(activation_fn())
        layers.append(nn.Dropout(dropout_rate))
        for i in range(len(hidden_dims) - 1):
            layers.append(nn.Linear(hidden_dims[i], hidden_dims[i + 1]))
            layers.append(activation_fn())
            layers.append(nn.Dropout(dropout_rate))
        layers.append(nn.Linear(hidden_dims[-1], output_dim))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

    def train_model(self, X_train, y_train, X_val, y_val, num_epochs, patience, lr):
        criterion = nn.MSELoss()
        optimizer = optim.Adam(self.parameters(), lr=lr)
        best_loss = float('inf')
        best_model = None
        epochs_no_improve = 0

        for epoch in range(num_epochs):
            self.train()
            optimizer.zero_grad()
            outputs = self(X_train)
            loss = criterion(outputs, y_train)
            loss.backward()
            optimizer.step()

            self.eval()
            with torch.no_grad():
                val_outputs = self(X_val)
                val_loss = criterion(val_outputs, y_val)

            if val_loss < best_loss:
                best_loss = val_loss
                best_model = self.state_dict()
                epochs_no_improve = 0
            else:
                epochs_no_improve += 1

            if epochs_no_improve == patience:
                break

        self.load_state_dict(best_model)

# Load the data
df_1 = pd.read_csv('training_data.csv')

# Split the data into features and target
X = df_1[['zeta', 'C_proc', 'RPM_Average', 'pitch', 'roll', 'yaw']].values
y = np.array([ast.literal_eval(lws) for lws in df_1['Lws']])

# Standardize the features using MinMaxScaler
scaler = MinMaxScaler(feature_range=(1, 2))
X = scaler.fit_transform(X)

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Convert data to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

input_dim = X_train.shape[1]
output_dim = y_train.shape[1]

# Objective function for Optuna
def objective(trial: optuna.Trial):
    # Hyperparameters to tune
    num_layers = trial.suggest_int('num_layers', 1, 6, step=1)
    hidden_dims = []
    layer_dim_increment = trial.suggest_int('layer_dim_increment', 32, 512, step=32)
    trial.set_user_attr("layer_dim_increment", layer_dim_increment)
    for i in range(num_layers):
        if i == 0:
            hidden_dims.append(trial.suggest_int(f'hidden_dim_{i}', input_dim, input_dim + layer_dim_increment, step=1))
        else:
            hidden_dims.append(trial.suggest_int(f'hidden_dim_{i}', hidden_dims[i-1] + 1, hidden_dims[i-1] + layer_dim_increment, step=1))
    dropout_rate = trial.suggest_float('dropout_rate', 0, 0.5, step=0.1)
    lr = trial.suggest_float('lr', 1e-5, 1e-1, log=True)
    activation_fn_name = trial.suggest_categorical('activation_fn', ['ReLU', 'LeakyReLU', 'Tanh'])
    num_epochs = 10000
    patience = 500

    # Map the activation function name to the actual function
    activation_fn = getattr(nn, activation_fn_name)

    model = DNN(input_dim, hidden_dims, output_dim, dropout_rate, activation_fn)

    # Train the model
    model.train_model(X_train, y_train, X_val, y_val, num_epochs, patience, lr)

    # Evaluate the model
    model.eval()
    with torch.no_grad():
        y_pred = model(X_test)
        actual_Lw_total = y_test.numpy().mean(axis=1)
        predicted_Lw_total = y_pred.numpy().mean(axis=1)
        mape = np.mean(np.abs((actual_Lw_total - predicted_Lw_total)) / actual_Lw_total) * 100

    return mape

# Run the optimization
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=1500)


[I 2024-12-12 16:49:29,060] A new study created in memory with name: no-name-be535d33-2f55-46a8-bc04-49dd794c1e3c
[I 2024-12-12 16:49:35,249] Trial 0 finished with value: 14.125236868858337 and parameters: {'num_layers': 4, 'layer_dim_increment': 448, 'hidden_dim_0': 68, 'hidden_dim_1': 130, 'hidden_dim_2': 141, 'hidden_dim_3': 434, 'dropout_rate': 0.4, 'lr': 7.713147108053869e-05, 'activation_fn': 'ReLU'}. Best is trial 0 with value: 14.125236868858337.
[I 2024-12-12 16:49:56,191] Trial 1 finished with value: 7.917112112045288 and parameters: {'num_layers': 3, 'layer_dim_increment': 416, 'hidden_dim_0': 297, 'hidden_dim_1': 603, 'hidden_dim_2': 605, 'dropout_rate': 0.1, 'lr': 0.046474353082588675, 'activation_fn': 'ReLU'}. Best is trial 1 with value: 7.917112112045288.
[I 2024-12-12 16:50:20,819] Trial 2 finished with value: 10.359404981136322 and parameters: {'num_layers': 4, 'layer_dim_increment': 32, 'hidden_dim_0': 13, 'hidden_dim_1': 14, 'hidden_dim_2': 23, 'hidden_dim_3': 32, 'd

In [16]:
# Print the best hyperparameters
print('Best hyperparameters: ', study.best_params)

Best hyperparameters:  {'num_layers': 2, 'layer_dim_increment': 32, 'hidden_dim_0': 13, 'hidden_dim_1': 40, 'dropout_rate': 0.0, 'lr': 0.014688982696300567, 'activation_fn': 'LeakyReLU'}
